# Finding open security groups with the AWS API

AWS API Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/index.html

In this exercise we'll start digging into the AWS API and learn how to find security groups that are open to the world and the EC2 instances that use them.

Steps:
1) Find all open security groups and output them in an easy-to-use format
2) Find of the security groups used by running instances
3) Check a whitelist to see whether each instance should be allowed to use an open security group
4) Disable non-whitelisted security groups 
5) Send a notification to be logged in Splunk

In [1]:
# First we'll import boto3 to be used for everything else
import boto3

## describe_security_groups
First we'll take a look at what visibility AWS offers us into security group configurations. The `describe_security_group` endpoint can return information for all security groups, giving us a good starting point.

Documentation: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.describe_security_groups

In [11]:
def open_security_groups():
    """Return all security groups that are allow inbound connections from the """
    ec2_client = boto3.client("ec2")
    security_groups = ec2_client.describe_security_groups(
        Filters=[
            {"Name": "ip-permission.cidr", "Values": ["0.0.0.0/0"]},
        ]
    )
    groups_whitelist = ["allow_splunk_ports_ingress"]
    ports_whitelist = [22, 80, 443, 8080, 8000]
    test_failsafe = "despicable"
    open_groups = []
    for sg in security_groups["SecurityGroups"]:
        for permission in sg["IpPermissions"]:
            if sg["GroupName"] not in groups_whitelist and permission["ToPort"] not in ports_whitelist:
                if test_failsafe not in sg["GroupName"]:
                    # Make sure we only match a group with "despicable" in its name, just in case...
                    continue
                open_groups.append(sg["GroupId"])
    return list(set(open_groups))

open_security_groups()

['sg-00a2bf4d69d4e820e']

In [16]:
def instance_security_groups():
    """Return a summary of all the security group IDs assocated with running instances.
    
    This format will look like: 
        {'i-096e3b9655241f365': ['sg-05777ecea90c47aae'], ...}
    """
    ec2_client = boto3.client("ec2")
    running_instances = ec2_client.describe_instances(
        Filters=[
            {"Name": "instance-state-name", "Values": ["running", "stopped"]},
        ]
    )
    instances = {}
    for reservation in running_instances["Reservations"]:
        for instance in reservation["Instances"]:
            instance_id = instance["InstanceId"]
            for iface in instance["NetworkInterfaces"]:
                instances[instance_id] = []
                for group in iface["Groups"]:
                    instances[instance_id].append(group["GroupId"])
    return instances
    
instance_security_groups()

{'i-000eabeee40b6ad02': ['sg-04937680a53790fd3'],
 'i-0ab2965541ab92819': ['sg-0e47ff803f308e638',
  'sg-eeccb096',
  'sg-00a2bf4d69d4e820e']}

Now that we have easier-to-parse lists of all security groups and the security groups our instances use, we can go through them and easily find which instances are using open security groups.

In [17]:
open_groups = open_security_groups()
instance_groups = instance_security_groups()
for instance, groups in instance_groups.items():
    instance_open_groups = list(set(groups).intersection(open_groups))
    if instance_open_groups:
        print(instance, instance_open_groups)

i-0ab2965541ab92819 ['sg-00a2bf4d69d4e820e']


Now that we know we can identify instances with open groups, let's write the code to remove that open group

In [41]:
def remove_security_group(instance_id, sg_id):
    ec2 = boto3.client('ec2')
    group_name = 'default'
    response = ec2.describe_security_groups(
        Filters=[
            dict(Name='group-name', Values=[group_name])
        ]
    )
    default_group_id = response['SecurityGroups'][0]['GroupId']
    ec2_resource = boto3.resource("ec2")
    instance = ec2_resource.Instance(instance_id)
    new_groups = [g["GroupId"] for g in instance.security_groups if g["GroupId"] != sg_id]
    # Security groups can't be empty, so if this list is empty use the default security group
    if not new_groups:
        new_groups = [default_group_id]
    instance.modify_attribute(Groups=new_groups)

Now we can put it all together:

In [19]:
def remediate_open_security_groups():
    open_groups = open_security_groups()
    instance_groups = instance_security_groups()
    for instance, groups in instance_groups.items():
        instance_open_groups = list(set(groups).intersection(open_groups))
        for group in instance_open_groups:
            print(f"Removing {group} from {instance}")
            remove_security_group(instance, group)

remediate_open_security_groups()

Removing sg-00a2bf4d69d4e820e from i-0ab2965541ab92819


sg-eeccb096
